### Veeam Backup for Azure job update

This Notebook shows how you can perform updates to multiple policies in Veeam Backup for Azure. 

First as this is Python we need to import all the packages, note that requests needs to be installed. 

    pip install requests

This is all based on the Veeam helpcenter: https://helpcenter.veeam.com/docs/vbazure/rest/editing_backup_policy_by_policy_id.html?ver=10

In [279]:
import requests
import urllib3
import pprint
import getpass
urllib3.disable_warnings()

Enter the VBA credentials

In [280]:
ip = input('Enter VBA IP: ')
url = f"https://{ip}/api/oauth2/token"
username = input('Enter Username: ')
password = getpass.getpass("Enter password: ")

Enter VBA IP:  11.11.11.11
Enter Username:  test
Enter password:  ······


Log into VBA and get the access token, this requires building a body object (data) and the header object.

In [272]:
data = {"grant_type" : "password", "username" : username, "password": password}
headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}

res = requests.post(url, data=data, verify=False)

print(f"Status code: {res.status_code}")
res_json = res.json()
access_token = res_json.get('access_token')
# print(res_json)

Status code: 200


Add the access token to the header

In [273]:
headers['Authorization'] = "Bearer " + access_token

Make request to get policies

In [274]:
export_policies = f'https://{ip}/api/v1/policies/export'

res = requests.post(export_policies, headers=headers, verify=False).json()

# policy = res['results'][0]

pprint.pprint(res)

{'_links': {'self': {'href': 'https://51.140.176.204:443/api/v1/policies?usn=0&offset=0&limit=-1'}},
 'limit': -1,
 'offset': 0,
 'results': [{'_links': {'excludeditems': {'href': 'https://51.140.176.204/api/v1/policies/7c8a6e8b-af06-4f76-8d32-c9cdd5423b18/excludedItems'},
                         'selecteditems': {'href': 'https://51.140.176.204/api/v1/policies/7c8a6e8b-af06-4f76-8d32-c9cdd5423b18/selectedItems'},
                         'self': {'href': 'https://51.140.176.204/api/v1/policies/7c8a6e8b-af06-4f76-8d32-c9cdd5423b18'},
                         'targetrepository': {'href': 'https://51.140.176.204/api/v1/repositories/1'}},
              'backupSettings': {'retentionSettings': {'retentionDurationType': 'Daily',
                                                       'timeRetentionDuration': 10},
                                 'targetRepositoryId': '1'},
              'backupType': 'SelectedItems',
              'description': '',
              'excludedItemsCount': 0,
   

Need to grab all the relevant attributes that we will need including, name, id and tenantId and construct a new object to use to update the policy.

Also going to build a list of the policy ids to iterate over, I could use object above but it's just easier to create a new list.

In [275]:
attributes = []

for item in res['results']:
    name = item['name']
    ident = item['id']
    tenantId = item['tenantId']
    new_object= {'name': name, 'id': ident, 'tendantId': tenantId}
    attributes.append(new_object)

policy_ids = [item['id'] for item in res['results']]
print(attributes)

[{'name': 'First', 'id': '7c8a6e8b-af06-4f76-8d32-c9cdd5423b18', 'tendantId': '92b77594-14f3-405e-9edc-f507fab9cad1'}]


We also need to get a list of VMs in each policy, to do this we need to send a new GET request to the 'selectedItems' end point then add that to the above object.

In [276]:
policy_vms = []
policy_num = 0

for policy in policy_ids:
    ps = requests.get(f'https://{ip}/api/v1/policies/{policy}/selectedItems', headers=headers, verify=False).json()
    pvms = [item['virtualMachine']['id'] for item in ps['results']]
    vm_list = []
    for vms in pvms:
        vm_list.append({'id': vms})
    attributes[policy_num]['virtualMachines'] = vm_list
    policy_num += 1

pprint.pprint(attributes)

[{'id': '7c8a6e8b-af06-4f76-8d32-c9cdd5423b18',
  'name': 'First',
  'tendantId': '92b77594-14f3-405e-9edc-f507fab9cad1',
  'virtualMachines': [{'id': 'fpfopfpera43g4f4uxf9mqhj9kuj7qpzyc6fsi99k8tjgb14t1oy'}]}]


Next we will update the template policy that was copied from VBA and update it with each of the new updated policies. Each of them will be held in a list of dictionaries.

Note that this only shows a backup, no snapshots are configured in this policy.

In [277]:
# Policy Template
policy_update = {
    "name": "",
    "description": "",
    "isEnabled": True,
    "priority": 1,
    "tenantId": "",
    "backupType": "SelectedItems",
    "regions": [
        {
            "regionId": "uksouth"
        }
    ],
    "selectedItems": {
        "subscriptions": [],
        "resourceGroups": [],
        "tags": [],
        "virtualMachines": [
            {
                "id": ""
            }
        ]
    },
    "excludedItems": {
        "virtualMachines": []
    },
    "backupSettings": {
        "targetRepositoryId": "1",
        "retentionSettings": {
            "retentionDurationType": "Daily",
            "timeRetentionDuration": 4
        },
        "schedule": {
            "frequencyType": "Daily",
            "dailyTime": "20:00:00",
            "dailyType": "Everyday",
            "selectedDays": [
                "Sunday",
                "Monday",
                "Tuesday",
                "Wednesday",
                "Thursday",
                "Friday",
                "Saturday"
            ]
        }
    },
    "retrySettings": {
        "retryCount": 3
    }
}

policies = []

# looping through each of the attribute objects and creating a new Policy object that we can use.
for item in attributes:
    policy_update['name'] = item['name']
    policy_update['tenantId'] = item['tendantId']
    policy_update['selectedItems']['virtualMachines'] = item['virtualMachines']
    policy_update['backupSettings']['retentionSettings']['timeRetentionDuration'] = 10 # making a single change to the retetnion
    policies.append(policy_update)
    
pprint.pprint(policies)

[{'backupSettings': {'retentionSettings': {'retentionDurationType': 'Daily',
                                           'timeRetentionDuration': 10},
                     'schedule': {'dailyTime': '20:00:00',
                                  'dailyType': 'Everyday',
                                  'frequencyType': 'Daily',
                                  'selectedDays': ['Sunday',
                                                   'Monday',
                                                   'Tuesday',
                                                   'Wednesday',
                                                   'Thursday',
                                                   'Friday',
                                                   'Saturday']},
                     'targetRepositoryId': '1'},
  'backupType': 'SelectedItems',
  'description': '',
  'excludedItems': {'virtualMachines': []},
  'isEnabled': True,
  'name': 'First',
  'priority': 1,
  'regions': [{'regionId': 'uks

Next loop through the policy ids sending a PUT request to the API to make the changes to the policies.

In [267]:
# Lists to hold the response codes and responses from the calls
status_codes = []
responses = []

# Loop through each of the attributes first as they hold the policy id
for item in attributes:
    policy_url = f'https://{ip}/api/v1/policies/{item['id']}'  
    # Next loop through the policies
    for policy in policies:
        # Where there is a matching name we send the request with the new policy dictionary converted to JSON (json = policy)
        if policy['name'] == item['name']:
            res = requests.put(policy_url, headers=headers, json=policy, verify=False)
            # Update each of the response arrays
            status_codes.append(res.status_code)
            responses.append(res)

for item in status_codes:
    if item == 204:
        print("Upate OK")
    else:
        print('Update Error')

print(responses)    


Upate OK
[<Response [204]>]


Finally you'll want to delete the headers with the access token

In [278]:
headers = []
print(headers)

[]


All done! See simple :)